In [2]:
""" !pip install datetime
!pip install astroquery """

  Obtaining dependency information for datetime from https://files.pythonhosted.org/packages/95/88/3b9d4042b396221a132180b392ab2a174031a6fb579f7927f3909fc183a7/DateTime-5.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 1.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 678.6 kB/s eta 0:00:00--:--


<p>
<img src="https://drive.google.com/uc?id=1ZHUKcwzA5c-PKUwoGmBG3TsHk6TmZ-Y0" align="left" width="250">
</p>
<p>
<h1><b><center>Mecánica Celeste</center></b></h1>
<h2><b><center>Prof. Jorge I. Zuluaga</center></b></h1>
<h2><b><center>Tarea 1</center></b><h2>
<h3><b><center>Constantes de movimiento del sistema solar</center></b><h3>
<h5><center><b>Asignada</b>: 15 de Agosto de 2023</center><h5>
<h5><center><b>Entrega</b>: 29 de Agosto de 2023</center><h5>
</p>

<hr/>
<b>Nombre</b>: Bryan Camilo Restrepo Arcila
<br/>
<b>Cédula</b>:1000567825
<br/>
<b>Última actualización</b>:
<hr/>

## Enunciado

**Objetivo**: El objetivo de esta tarea es poner a prueba los resultados vistos en clase sobre las constantes en el problema de los N-cuerpos, en este caso aplicados al sistema solar.  Para ello usaremos el sistema de datos del JPL Horizons y el paquete `astroquery`.

**Procedimiento**:

Para conseguir el objetivo:

1. Obtener las masas, posiciones y velocidades de los planetas del sistema solar (8 planetas) y del Sol durante 20 años (por ejemplo entre 2000 y 2020), cada 3 meses. Use para ello Astroquery y SPICE (para las masas).

2. Para cada punto en el tiempo calcular:

   - El momentum lineal de cada cuerpo.
   - El momentum angular de cada cuerpo.
   - La energía cinética de cada cuerpo.
   - La energía potencial gravidtacional de todo el sistema.

3. Usando el valor de cada una de las cantidades anteriores vamos a calcular el valor de las cuadraturas del sistema:

   - Calcular el momentum lineal total (3 cuadraturas).
   - Calcular el momentum angular total (3 cuadraturas).
   - Calcular la energía total total (1 cuadratura).

   Es muy importante tener en cuenta que el valor de cada una de las cuadraturas puede ser inmenso (las masas del sistema solar son inmensas). Por lo tanto, debemos dividir cada cuadratura por el valor característico de. para ello calcule el valor promedio del momentum lineal, angular y energía cinética del Sol.

4. Hacer gráficos de las 10 cuadraturas.

5. El momentum angular total del sistema solar define el Plano de Laplace del sistema. El momentum angular de cada uno de los cuerpos del sistema solar define la inclinación de la órbita osculatriz del cuerpo respecto al plano de Laplace. Calcular, para cada tiempo, el ángulo entre el plano de Laplace y la órbita osculatriz de cada cuerpo (incluyendo el Sol).

Ayuda: el ángulo entre dos vectores se calcula usando la propiedad del producto punto A⋅ B = A B cos θ.

6. Haga un gráfico del ángulo de la órbita osculatriz de cada cuerpo como función del tiempo.

7. Calcule el valor promedio de la inclinación orbital de cada planeta en grados respecto al plano invariante de Laplace.

**Resultados esperados**:

- Este notebook con el procedimiento debidamente comentado.

- Gráficos mínimos:
  - Gráfico de las 10 cuadraturas normalizadas (divididas por el valor para el Sol).

  - Gráfico de la inclinación de la órbita osculatriz de cada planeta y el Sol.

- Datos mínimos:
  - Valor de la inclinación orbital de cada planeta

- Gráficos opcionales:
  - Gráfico de las órbitas de los planetas.
  - Gráfico de las magnitudes del momentum lineal y angular total como función del tiempo.

- En caso de utilizar ayudas como `ChatGPT` o códigos de `Stack Over Flow` debe citar debidamente la fuente, pero también entender a cabalidad el código, el profesor puede pedir justificación.

## Solución

## 1. Obtener las masas, posiciones y velocidades de los planetas del sistema solar (8 planetas) y del Sol durante 20 años (por ejemplo entre 2000 y 2020), cada 3 meses. Use para ello Astroquery y SPICE (para las masas).

In [20]:
#Pimero importemos las librerias que vamos a utilizar

import pandas as pd
from astroquery.jplhorizons import Horizons
from astropy import constants
import datetime
import numpy as np
#----------------------------------------------------------------

#Extraigamos la información por medio del id de los astros
id_planets=list(np.arange(199,1199,100))
id_planets.remove(999) #Quitando a Pluton de la lista

for i in  id_planets:
    #Extrayendo los datos de los planetas en referencia al baricentro del sistema solar
    information = Horizons(id=str(i), location='0',epochs={'start':'2002-01-01', 'stop':'2022-12-31','step':'90d'}).vectors().to_pandas()[['datetime_str', 'x', 'y', 'z', 'vx', 'vy','vz']]
    print(information)
    
    

                      datetime_str         x         y         z        vx  \
0   A.D. 2002-Jan-01 00:00:00.0000  0.530696 -1.109092 -0.044066  0.022346   
1   A.D. 2002-Apr-01 00:00:00.0000  1.340539  0.106918 -0.039994 -0.002045   
2   A.D. 2002-Jun-30 00:00:00.0000  0.215629  0.980823 -0.034974 -0.020330   
3   A.D. 2002-Sep-28 00:00:00.0000 -0.652995 -0.047782 -0.029006 -0.006479   
4   A.D. 2002-Dec-27 00:00:00.0000  0.410044 -0.887308 -0.022213  0.004423   
..                             ...       ...       ...       ...       ...   
81  A.D. 2021-Dec-17 00:00:00.0000  0.115535 -1.367692 -0.050166  0.036896   
82  A.D. 2022-Mar-17 00:00:00.0000  1.232246 -0.423548 -0.051060  0.019185   
83  A.D. 2022-Jun-15 00:00:00.0000  0.386975  0.691400 -0.051189 -0.001248   
84  A.D. 2022-Sep-13 00:00:00.0000 -0.687366 -0.098801 -0.050370  0.010578   
85  A.D. 2022-Dec-12 00:00:00.0000  0.150022 -1.196671 -0.048667  0.027900   

          vy        vz  
0   0.030571  0.001769  
1   0.045745 

In [23]:
import pandas as pd
from astroquery.jplhorizons import Horizons
from astropy import constants
import datetime
import numpy as np

# Lista de IDs de los planetas
id_planets = list(np.arange(199, 1199, 100))
id_planets.remove(999)  # Quitando a Pluton de la lista

# Diccionario con los nombres de los planetas para una mejor identificación
planet_names = {
    199: 'Mercury',
    299: 'Venus',
    399: 'Earth',
    499: 'Mars',
    599: 'Jupiter',
    699: 'Saturn',
    799: 'Uranus',
    899: 'Neptune',
    1099:'sun'
}

# Inicializar el DataFrame final
df_planets_sun_information = None

for i in id_planets:
    # Extrayendo los datos de los planetas en referencia al baricentro del sistema solar
    information = Horizons(id=str(i), location='0', epochs={'start': '2002-01-01', 'stop': '2022-12-31', 'step': '90d'}).vectors().to_pandas()[['datetime_str', 'x', 'y', 'z', 'vx', 'vy', 'vz']]
    
    # Cambiar nombres de columnas para identificar a qué planeta pertenecen
    columns = {col: f"{planet_names[i]}_{col}" for col in information.columns if col != "datetime_str"}
    information = information.rename(columns=columns)
    
    # Si el DataFrame final está vacío, toca ponerle la información del primer planeta (Mercurio)
    if df_planets_sun_information is None:
        df_planets_sun_information = information
    else:
        # Combinar DataFrames basado en la columna datetime_str
        df_planets_sun_information = df_planets_sun_information.merge(information, on='datetime_str', how='outer')

    
df_planets_sun_information


,datetime_str,Mercury_x,Mercury_y,Mercury_z,Mercury_vx,Mercury_vy,Mercury_vz,Venus_x,Venus_y,Venus_z,...,Neptune_z,Neptune_vx,Neptune_vy,Neptune_vz,sun_x,sun_y,sun_z,sun_vx,sun_vy,sun_vz
0,A.D. 2002-Jan-01 00:00:00.0000,0.530696,-1.109092,-0.044066,0.022346,0.030571,0.001769,0.212419,-1.693480,-0.011990,...,0.056900,0.019798,0.005145,-0.000108,2.513070,-0.860100,-0.170810,0.016803,0.015581,0.002384
1,A.D. 2002-Apr-01 00:00:00.0000,1.340539,0.106918,-0.039994,-0.002045,0.045745,0.002182,1.386339,0.788867,-0.015286,...,0.048192,-0.000625,0.019080,-0.000163,3.055378,1.371170,0.053765,-0.008163,0.028174,0.002452
2,A.D. 2002-Jun-30 00:00:00.0000,0.215629,0.980823,-0.034974,-0.020330,0.026970,0.002717,-0.812528,0.742523,0.035047,...,0.039484,-0.014521,-0.000346,-0.000088,1.312797,3.058665,0.269841,-0.025297,0.005783,0.002236
3,A.D. 2002-Sep-28 00:00:00.0000,-0.652995,-0.047782,-0.029006,-0.006479,0.012027,0.003207,-0.299122,-0.275845,-0.043070,...,0.030807,0.004073,-0.015206,-0.000029,-0.380467,2.547107,0.447561,-0.008250,-0.012559,0.001766
4,A.D. 2002-Dec-27 00:00:00.0000,0.410044,-0.887308,-0.022213,0.004423,0.029887,0.003515,-0.365960,-0.422139,0.033694,...,0.022124,0.019949,0.003623,-0.000103,-0.208341,1.922182,0.573974,0.007433,0.003049,0.001103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,A.D. 2021-Dec-17 00:00:00.0000,0.115535,-1.367692,-0.050166,0.036896,0.012954,-0.000588,0.145686,-0.297853,-0.004007,...,-0.597598,0.017995,0.001678,-0.000068,-4.116896,-1.408166,0.236217,0.018031,-0.008563,-0.001409
82,A.D. 2022-Mar-17 00:00:00.0000,1.232246,-0.423548,-0.051060,0.019185,0.034486,-0.000294,0.414655,-0.501711,0.027363,...,-0.604292,0.001860,0.020516,-0.000141,-2.925602,-1.124027,0.105377,0.003509,0.010567,-0.001550
83,A.D. 2022-Jun-15 00:00:00.0000,0.386975,0.691400,-0.051189,-0.001248,0.021709,0.000159,0.838545,1.005474,-0.042001,...,-0.610940,-0.016620,0.005147,-0.000082,-3.552380,-0.636564,-0.029233,-0.013408,-0.004259,-0.001507
84,A.D. 2022-Sep-13 00:00:00.0000,-0.687366,-0.098801,-0.050370,0.010578,0.005135,0.000654,-1.575456,0.592414,0.039412,...,-0.617409,-0.002442,-0.013870,-0.000005,-4.268588,-1.998413,-0.162605,0.002269,-0.022469,-0.001390


In [10]:
from astroquery.jplhorizons import Horizons
from astropy.time import Time

# Define the object (Mars) and the time for which you want the ephemeris
obj = Horizons(id='499',  # 499 is the ID for Mars
               location=None,  # Geocentric location
               epochs=Time('2022-01-01').jd)  # Julian date for 2022-01-01

# Query the ephemeris
eph = obj.ephemerides()

eph

targetname,datetime_str,datetime_jd,solar_presence,flags,RA,DEC,RA_app,DEC_app,RA_rate,DEC_rate,AZ,EL,AZ_rate,EL_rate,sat_X,sat_Y,sat_PANG,siderealtime,airmass,magextinct,V,surfbright,illumination,illum_defect,sat_sep,sat_vis,ang_width,PDObsLon,PDObsLat,PDSunLon,PDSunLat,SubSol_ang,SubSol_dist,NPole_ang,NPole_dist,EclLon,EclLat,r,r_rate,delta,delta_rate,lighttime,vel_sun,vel_obs,elong,elongFlag,alpha,lunar_elong,lunar_illum,sat_alpha,sunTargetPA,velocityPA,OrbPlaneAng,constellation,TDB-UT,ObsEclLon,ObsEclLat,NPole_RA,NPole_DEC,GlxLon,GlxLat,solartime,earth_lighttime,RA_3sigma,DEC_3sigma,SMAA_3sigma,SMIA_3sigma,Theta_3sigma,Area_3sigma,RSS_3sigma,r_3sigma,r_rate_3sigma,SBand_3sigma,XBand_3sigma,DoppDelay_3sigma,true_anom,hour_angle,alpha_true,PABLon,PABLat
---,---,d,---,---,deg,deg,deg,deg,arcsec / h,arcsec / h,deg,deg,arcsec / min,arcsec / min,arcsec,arcsec,deg,---,---,mag,mag,mag / arcsec2,%,arcsec,arcsec,---,arcsec,deg,deg,deg,deg,deg,arcsec,deg,arcsec,deg,deg,AU,km / s,AU,km / s,min,km / s,km / s,deg,---,deg,deg,%,deg,deg,deg,deg,---,s,deg,deg,deg,deg,deg,deg,---,min,arcsec,arcsec,arcsec,arcsec,deg,arcsec2,arcsec,km,km / s,Hz,Hz,s,deg,---,deg,deg,deg
str10,str24,float64,str1,str1,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,float64,float64,float64,int64,int64,int64,float64,float64,float64,float64,float64,str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str2,float64,float64,float64,float64,float64,float64,float64,str3,float64,float64,float64,float64,float64,float64,float64,int64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64,int64,float64,float64,float64
Mars (499),2022-Jan-01 00:00:00.000,2459580.5,,,251.3392,-22.46119,251.65918,-22.49996,105.5744,-14.9821,--,--,--,--,-98712.37,1872.539,265.184,--,999,--,1.541,4.265,97.77674,0.0889,98748.37,*,4.000294,212.801951,4.676508,228.439509,12.185259,96.94,0.59,33.5585,1.982,235.6553,-0.1985,1.536649632621,-2.2249396,2.34115305967985,-9.0831555,19.47077627,23.9241663,51.1083877,27.4301,/L,17.1451,2.643,4.7342,135.4248,277.057,279.135,-0.60565,Oph,69.183891,253.0991162,-0.1319891,317.65809,52.8731,357.506007,14.824029,--,0.0,--,--,--,--,--,--,--,--,--,--,--,--,259.4218,--,17.1503,244.2258,-0.1656
